In [1]:
!pip install pymorphy2
!pip install python-docx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 43.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=80b16fde9319d16147c298271c95d9f23c00f31bc79732721c09999f46cce5ff
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 27.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184505 sha256=d3b1ca60b38b751c36f15398aa9a8426dbfcf0b4e369501fdefa4f7a066bded1
  Stored in directory: /root/.cac

In [2]:
import sklearn
import re
import pymorphy2
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
import docx

In [3]:
morph = pymorphy2.MorphAnalyzer()

def lemmatize(text):
    words = text.split() # разбиваем текст на слова
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)

    return res

In [4]:
txt=[
    "Отсутствие летальности в результате суицида",
    "Летальных случаев в результате суицида не было",
    "Летальность в результате суицида отсутствует",
    "Летальных случаев в результате суицида выявлено не было",
    "Показатель отсутствия летальности в результате суицида достигается",
    "В 2022 году было выявлено 3 случая летальных исходов в результате суицида",
    "В отчетном периоде были 4 случая летальности в результате суицида",
    "Показатель отсутствия летальности в результате суицида не достигается, поскольку был выявлен 1 случай",
    "Летальность в результате суицида присутствует, количество подобных случаев равняется 2-ум",
    "В отчетном периоде произошло 3 случая летальности в результате суицида"
]

In [5]:
posneg = [
    "pos",
    "pos",
    "pos",
    "pos",
    "pos",
    "neg",
    "neg",
    "neg",
    "neg",
    "neg"
]

In [6]:
txt_l=[]
for i in txt:
  words = i.split(" ")
  words = filter(lambda x: len(re.findall(r"[0-9]", x)) == 0,words)
  i = " ".join(words)
  i=i.replace(",","")
  i=i.replace(".","")
  i=i.replace("-","")
  i=i.lower()
  txt_l.append(lemmatize(i))
txt_l

[['отсутствие', 'летальность', 'в', 'результат', 'суицид'],
 ['летальный', 'случай', 'в', 'результат', 'суицид', 'не', 'быть'],
 ['летальность', 'в', 'результат', 'суицид', 'отсутствовать'],
 ['летальный', 'случай', 'в', 'результат', 'суицид', 'выявить', 'не', 'быть'],
 ['показатель',
  'отсутствие',
  'летальность',
  'в',
  'результат',
  'суицид',
  'достигаться'],
 ['в',
  'год',
  'быть',
  'выявить',
  'случай',
  'летальный',
  'исход',
  'в',
  'результат',
  'суицид'],
 ['в',
  'отчётный',
  'период',
  'быть',
  'случай',
  'летальность',
  'в',
  'результат',
  'суицид'],
 ['показатель',
  'отсутствие',
  'летальность',
  'в',
  'результат',
  'суицид',
  'не',
  'достигаться',
  'поскольку',
  'быть',
  'выявить',
  'случай'],
 ['летальность',
  'в',
  'результат',
  'суицид',
  'присутствовать',
  'количество',
  'подобный',
  'случай',
  'равняться'],
 ['в',
  'отчётный',
  'период',
  'произойти',
  'случай',
  'летальность',
  'в',
  'результат',
  'суицид']]

In [7]:
words_set = set()
[[words_set.add(i2) for i2 in i] for i in txt_l]
words_set

{'быть',
 'в',
 'выявить',
 'год',
 'достигаться',
 'исход',
 'количество',
 'летальность',
 'летальный',
 'не',
 'отсутствие',
 'отсутствовать',
 'отчётный',
 'период',
 'подобный',
 'показатель',
 'поскольку',
 'присутствовать',
 'произойти',
 'равняться',
 'результат',
 'случай',
 'суицид'}

In [8]:
words_dict = {}
for i in words_set:
  words_dict [i] = 0
for i in txt_l:
  for i2 in i:
    words_dict[i2]+=1
words_dict

{'равняться': 1,
 'выявить': 3,
 'достигаться': 2,
 'в': 13,
 'количество': 1,
 'присутствовать': 1,
 'показатель': 2,
 'результат': 10,
 'летальность': 7,
 'исход': 1,
 'поскольку': 1,
 'суицид': 10,
 'летальный': 3,
 'случай': 7,
 'подобный': 1,
 'быть': 5,
 'период': 2,
 'отсутствие': 3,
 'отсутствовать': 1,
 'год': 1,
 'произойти': 1,
 'не': 3,
 'отчётный': 2}

In [9]:
words = []
for i in words_dict:
  if words_dict[i]<7:
    words.append(i)
words

['равняться',
 'выявить',
 'достигаться',
 'количество',
 'присутствовать',
 'показатель',
 'исход',
 'поскольку',
 'летальный',
 'подобный',
 'быть',
 'период',
 'отсутствие',
 'отсутствовать',
 'год',
 'произойти',
 'не',
 'отчётный']

In [10]:
def str_to_array(i):
  words_list = i.split(" ")
  words_list = filter(lambda x: len(re.findall(r"[0-9]", x)) == 0,words_list)
  i = " ".join(words_list)
  i=i.replace(",","")
  i=i.replace(".","")
  i=i.replace("-","")
  i=i.lower()
  wa = lemmatize(i)
  words_set = set()
  [words_set.add(i) for i in wa]
  words_dict = {}
  for i in words_set:
    words_dict [i] = 0
  for i in wa:
      words_dict[i]+=1
  #print(words_dict)
  result = []
  for i in words:
    if i in words_dict:
      result.append(words_dict[i])
    else:
      result.append(0)
  return result
  #print(result)
X = []
for i in txt:
  X.append(str_to_array(i))
print(X)
Y = []
for i in posneg:
  if i == "pos":
    Y.append([1,0])
  else:
    Y.append([0,1])
print(Y)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0], [0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1], [0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0], [1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1]]
[[1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [0, 1], [0, 1], [0, 1]]


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

model = Sequential()

model.add(Dense(10, input_dim=18, activation='relu'))

model.add(Dense(10, activation='sigmoid'))

model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=1000, validation_data=(X_test, y_test))


Epoch 1/1000
1/1 [==============================] - 1s 1s/step - loss: 0.8094 - accuracy: 0.3750 - val_loss: 0.4373 - val_accuracy: 1.0000
Epoch 2/1000
1/1 [==============================] - 0s 97ms/step - loss: 0.8044 - accuracy: 0.3750 - val_loss: 0.4420 - val_accuracy: 1.0000
Epoch 3/1000
1/1 [==============================] - 0s 88ms/step - loss: 0.7994 - accuracy: 0.3750 - val_loss: 0.4467 - val_accuracy: 1.0000
Epoch 4/1000
1/1 [==============================] - 0s 62ms/step - loss: 0.7945 - accuracy: 0.3750 - val_loss: 0.4514 - val_accuracy: 1.0000
Epoch 5/1000
1/1 [==============================] - 0s 79ms/step - loss: 0.7897 - accuracy: 0.3750 - val_loss: 0.4562 - val_accuracy: 1.0000
Epoch 6/1000
1/1 [==============================] - 0s 81ms/step - loss: 0.7849 - accuracy: 0.3750 - val_loss: 0.4609 - val_accuracy: 1.0000
Epoch 7/1000
1/1 [==============================] - 0s 79ms/step - loss: 0.7802 - accuracy: 0.3750 - val_loss: 0.4657 - val_accuracy: 1.0000
Epoch 8/1000
1/

In [12]:
def predict(text):
  input_data = np.array([str_to_array(text)])
  prediction = model.predict(input_data,verbose = 0)
  print("Текст: "+text)
  if prediction[0][0] > prediction[0][1]:
    print("Позитив")
    print("Штраф 0 баллов")
  else:
    print("Негатив")
    print("Штраф "+ str(int(min(re.findall(r"[0-9]+",text))) if int(min(re.findall(r"[0-9]+",text))) <= 10 else 10)+" баллов")
for i in txt:
  predict(i)

Текст: Отсутствие летальности в результате суицида
Позитив
Штраф 0 баллов
Текст: Летальных случаев в результате суицида не было
Позитив
Штраф 0 баллов
Текст: Летальность в результате суицида отсутствует
Позитив
Штраф 0 баллов
Текст: Летальных случаев в результате суицида выявлено не было
Позитив
Штраф 0 баллов
Текст: Показатель отсутствия летальности в результате суицида достигается
Позитив
Штраф 0 баллов
Текст: В 2022 году было выявлено 3 случая летальных исходов в результате суицида
Негатив
Штраф 10 баллов
Текст: В отчетном периоде были 4 случая летальности в результате суицида
Негатив
Штраф 4 баллов
Текст: Показатель отсутствия летальности в результате суицида не достигается, поскольку был выявлен 1 случай
Негатив
Штраф 1 баллов
Текст: Летальность в результате суицида присутствует, количество подобных случаев равняется 2-ум
Негатив
Штраф 2 баллов
Текст: В отчетном периоде произошло 3 случая летальности в результате суицида
Негатив
Штраф 3 баллов


In [13]:
# Открываем документ Word
document = docx.Document(input("Введите путь к файлу: "))

# Задаем искомую строку
search_text = 'Отсутствие летальности в результате суицида'

# Перебираем все таблицы в документе
for table in document.tables:
    # Перебираем все строки в каждой таблице
    for row in table.rows:
        # Перебираем все ячейки в каждой строке
        for cell in row.cells:
            # Проверяем, содержит ли данная ячейка искомую подстроку
            if search_text in cell.text:
                predict(row.cells[-1].text)

Введите путь к файлу: /content/СПб ПБСТИН.docx
Текст: В 2022 году было выявлено 2 случая летальных исходов в результате суицида.
Негатив
Штраф 2 баллов
